In [1]:
from numbalsoda import lsoda_sig, lsoda, dop853
from numba import njit, cfunc
import numpy as np

In [9]:
@cfunc(lsoda_sig)
def rhs(t, u, du, p):
    # Lotka-Volterra equations
    alpha, delta = p[0], p[1]
    beta, gamma = 1, 1
    prey, predator = u[0], u[1]  # x, y
    du[0] = prey * (alpha - beta * predator)  # dprey
    du[1] = predator * (-gamma + delta * prey)  # dpredator

funcptr = rhs.address # address to ODE function
u0 = np.array([1.0, 0.5]) # Initial conditions
data = np.array([1.0, 1.0]) # data you want to pass to rhs (alpha, delta parameters).
t_eval = np.array([1.1, 2.4, 3.9, 5.6, 7.5, 9.6, 11.9, 14.4])

# integrate with lsoda method
usol, success = lsoda(funcptr, u0, t_eval, data = data)

In [8]:
usol.shape

(1000, 2)

In [17]:
# timeit by repeatedly running the integration
import timeit
def run_integration():
    usol, success = lsoda(funcptr, u0, t_eval, data=data)

In [11]:
# Implement lotka volterra with solve_ivp
from scipy.integrate import solve_ivp

def lotka_volterra(t, u, alpha, delta):
    prey, predator = u
    du_dt = [prey * (alpha - prey * predator), predator * (-delta + prey * predator)]
    return du_dt

# Initial conditions and parameters
alpha, delta = 1.0, 1.0

def run_solve_ivp():
    sol = solve_ivp(lotka_volterra, [0, 15], u0, args=(alpha, delta), t_eval=t_eval)
    return sol

In [29]:
time_taken = timeit.timeit(run_solve_ivp, number=1000)
print(f"Time taken for 10000 runs: {time_taken:.4f} seconds")

time_taken = timeit.timeit(run_integration, number=1000)
print(f"Time taken for 10000 runs: {time_taken:.4f} seconds")

Time taken for 10000 runs: 0.4087 seconds
Time taken for 10000 runs: 0.0378 seconds
Time taken for 10000 runs: 0.4255 seconds
